In [2]:
import nltk
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [4]:
ModelName = 'google/flan-t5-small'
Dataset = 'yahoo_answers_qa'
input_prefix = 'Please answer this question:'

In [5]:
nltk.download('punkt', quiet=True)
metric = evaluate.load('rouge')

In [6]:
tokenizer = T5Tokenizer.from_pretrained(ModelName)
model = T5ForConditionalGeneration.from_pretrained(ModelName)
data_collator = DataCollatorForSeq2Seq(tokenizer, model)

c:\Users\mhmd5\anaconda3\envs\generalEnv3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
yahoo_qa_dataset = load_dataset(Dataset)
yahoo_qa_dataset = yahoo_qa_dataset['train'].train_test_split(test_size = 0.3)
yahoo_qa_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 61153
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 26209
    })
})

In [16]:
# train_yahoo_qa_dataset = yahoo_qa_dataset['train'].select(range(500))
# test_yahoo_qa_dataset = yahoo_qa_dataset['test'].select(range(400))

In [27]:
def preprocess_input(examples):

    inputs = [input_prefix + question for question in examples['question']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    labels = tokenizer(examples['answer'], max_length=512, truncation=True)
    model_inputs['labels'] = labels['input_ids']

    return model_inputs

tokenized_dataset = yahoo_qa_dataset.map(preprocess_input, batched= True)

Map:   0%|          | 0/61153 [00:00<?, ? examples/s]

Map:   0%|          | 0/26209 [00:00<?, ? examples/s]

c:\Users\mhmd5\anaconda3\envs\generalEnv3.11\Lib\site-packages\transformers\models\t5\tokenization_t5.py:303: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [19]:
def compute_metrics(eval_preds):
    
    preds, labels = eval_preds
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions = decoded_preds, reference = decoded_labels, use_stemmer = True)
    return result

In [23]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
# PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="Results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
#    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [28]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [29]:
trainer.train()

  0%|          | 0/22935 [00:00<?, ?it/s]

KeyboardInterrupt: 